In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys


import functions.parse_data as parse
import functions.handy_functions as hf
import torch.nn as nn
import torch

from multivariate_quantile_regression.network_model import QuantileNetwork

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

from tqdm import tqdm

import os
import random

/tmp/ipykernel_156141/3911021086.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-02-23 11:59:18.746961: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-23 11:59:18.781922: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-23 11:5

In [2]:
# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # CUDA is available, so let's set default device to GPU
    torch.set_default_device(0)
    print("CUDA is available. Using GPU.")
else:
    # CUDA is not available, so let's use the CPU
    print("CUDA is not available. Using CPU.")

# Example usage:
tensor = torch.randn(3, 3)  # Create a tensor on the selected device
print("Tensor is on device:", tensor.device)

CUDA is available. Using GPU.
Tensor is on device: cuda:0


In [3]:
channel_labels= ['Cloud_B02','Cloud_B03','Cloud_B04','Cloud_B05','Cloud_B06','Cloud_B07',
                 'Cloud_B08','Cloud_B08A','Cloud_B9','Cloud_B10','Cloud_B11','Cloud_B12']

data_water=parse.parse('SMHIdata/cloudrm_water.dat')
data_clear=parse.parse('SMHIdata/cloudrm_clear.dat')
data_ice=parse.parse('SMHIdata/cloudrm_ice.dat')
data_mixed=parse.parse('SMHIdata/cloudrm_mixed.dat')

#Concatinate all datasets
data_all=pd.concat([data_water, data_clear, data_ice, data_mixed])
data_all=data_all.drop(columns=['Surface_Desc','Cloud_B01','Clear_B01'])
df_truth=data_all.copy()

In [4]:
##Train test validation split##
X_labels= ['Cloud_B02','Cloud_B03','Cloud_B04','Cloud_B05','Cloud_B06',
           'Cloud_B07','Cloud_B08','Cloud_B08A','Cloud_B09','Cloud_B10','Cloud_B11','Cloud_B12',
           'Sat_Zenith_Angle','Sun_Zenith_Angle','Azimuth_Diff_Angle']

#Leave out 'GOT', 'Water_Vapor'
#Band 1 no go.
y_labels=['Clear_B02','Clear_B03','Clear_B04','Clear_B05','Clear_B06',
           'Clear_B07','Clear_B08','Clear_B08A','Clear_B09','Clear_B10','Clear_B11','Clear_B12']

X_truth=df_truth[X_labels]

##Split data##
X=data_all[X_labels]
y=data_all[y_labels]

X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.05)

X_test_truth=X_truth.iloc[X_test.index] #Save truth values without normalisation

In [5]:
quantiles=np.array([0.1,0.5,0.9])

sequence= lambda: nn.Sequential(
    nn.Linear(len(X_labels),64),
    nn.ReLU(),
    nn.Linear(64,64),
    nn.ReLU(),
    nn.Linear(64,64),
    nn.ReLU(),
    nn.Linear(64,64),
    nn.ReLU(),
    nn.Linear(64, len(quantiles)*len(y_labels)) #Output dimesion is number of quantiles times number of target variables
)

In [9]:
val_size=0.05
batch_size=500 

lr_s=[0.01, 0.001, 0.0001]

num_models=5 #Set number of ensambles
batch_size=500
nepochs=300
lr=0.001
data_norm=False
noise_ratio = 0.03

looped_metric="Loop_learning_rate"
main_filepath='pytorch_models/'+looped_metric

model_metrics=pd.DataFrame(columns=['Ensemble_mean','Ensemble_index',looped_metric,'MSE','PSNR','R2_score','Mean_Quantile_Loss','Quant_Rate','Quantiles'])
for lr in lr_s:

    
    models=[QuantileNetwork(quantiles=quantiles) for _ in range(num_models)]
    preds_total=[]
    for i,model in enumerate(models): 

        validation_indices=np.array(random.sample(range(len(X_train['Cloud_B02'])), int(len(X_train['Cloud_B02'])*val_size)))
        train_indices=[i for i in range(len(X_train['Cloud_B02'])) if np.any(validation_indices==i)==False]       

        model.fit(X_train.to_numpy(),y_train.to_numpy(), 
            train_indices, 
            validation_indices, 
            batch_size=batch_size,
            nepochs=nepochs,
            sequence=sequence(),
            lr=lr,
            data_norm=data_norm,
            noise_ratio=noise_ratio)
        
        filepath=main_filepath+'/'+str(lr)+'/model'+str(i)
        os.makedirs(filepath,exist_ok=True)
        torch.save(model,filepath+'/model_file')

        preds = model.predict(X_test.to_numpy())
        if i==0:
            preds_total=preds
        else:
            preds_total=preds_total+preds

        mse=mean_squared_error(y_test.to_numpy(),preds[:,:,1])
        psnr=QuantileNetwork.PSNR(y_test,preds[:,:,1])
        r2=r2_score(y_test.to_numpy(),preds[:,:,1])
        mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy(),preds,quantiles)
        quant_rate=QuantileNetwork.quant_rate(y_test.to_numpy(),preds)

        tmp_metrics=pd.DataFrame(data=[[False,i,lr,mse,psnr,r2,mean_quantile,quant_rate,quantiles]],columns=['Ensemble_mean','Ensemble_index',looped_metric,'MSE','PSNR','R2_score','Mean_Quantile_Loss','Quant_Rate','Quantiles'])
        model_metrics=pd.concat([model_metrics,tmp_metrics])
    
    preds_total=preds_total/num_models

    mse=mean_squared_error(y_test.to_numpy(),preds_total[:,:,1])
    psnr=QuantileNetwork.PSNR(y_test,preds_total[:,:,1])
    r2=r2_score(y_test.to_numpy(),preds_total[:,:,1])
    mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy(),preds_total,quantiles)
    quant_rate=QuantileNetwork.quant_rate(y_test.to_numpy(),preds_total)

    tmp_metrics=pd.DataFrame(data=[[True,np.nan,lr,mse,psnr,r2,mean_quantile,quant_rate,quantiles]],columns=['Ensemble_mean','Ensemble_index',looped_metric,'MSE','PSNR','R2_score','Mean_Quantile_Loss','Quant_Rate','Quantiles'])
    model_metrics=pd.concat([model_metrics,tmp_metrics])


X_test.to_csv(main_filepath+'/xtest.csv',index=False)
y_test.to_csv(main_filepath+'/ytest.csv',index=False)

model_metrics=model_metrics.reset_index(drop=True)
model_metrics.to_csv(main_filepath+'/model_metrics.csv',index=False)

Epoch 1


Batch number: 100%|██████████| 361/361 [00:00<00:00, 369.21it/s]

Training loss [1.7121707] Validation loss [1.4037145]
----New best validation loss---- [1.4037145]
Epoch 2



Batch number: 100%|██████████| 361/361 [00:00<00:00, 382.73it/s]

Training loss [1.2202023] Validation loss [1.1467676]
----New best validation loss---- [1.1467676]
Epoch 3


...


Batch number: 100%|██████████| 361/361 [00:00<00:00, 420.92it/s]

Training loss [0.67360467] Validation loss [0.6350559]
Epoch 300



Batch number: 100%|██████████| 361/361 [00:00<00:00, 421.49it/s]

Training loss [0.67284006] Validation loss [0.633391]


Best model out of total max epochs found at epoch 295


In [10]:
model_metrics

,Ensemble_mean,Ensemble_index,Loop_learning_rate,MSE,PSNR,R2_score,Mean_Quantile_Loss,Quant_Rate,Quantiles
0,False,0,0.0100,0.007358,26.042524,0.894771,0.552978,"[0.07755833333333334, 0.454775, 0.925633333333...","[0.1, 0.5, 0.9]"
1,False,1,0.0100,0.006965,26.280956,0.897168,0.542747,"[0.07836666666666667, 0.482625, 0.911366666666...","[0.1, 0.5, 0.9]"
2,False,2,0.0100,0.007261,26.100353,0.892764,0.554140,"[0.09786666666666667, 0.5122583333333334, 0.93...","[0.1, 0.5, 0.9]"
3,False,3,0.0100,0.007637,25.881205,0.889703,0.569278,"[0.11070833333333334, 0.5023083333333334, 0.93...","[0.1, 0.5, 0.9]"
4,False,4,0.0100,0.007040,26.234630,0.896178,0.535612,"[0.13103333333333333, 0.4884, 0.9251083333333333]","[0.1, 0.5, 0.9]"
5,True,NaN,0.0100,0.006541,26.553725,0.905769,0.510995,"[0.07778333333333333, 0.49714166666666665, 0.9...","[0.1, 0.5, 0.9]"
6,False,0,0.0010,0.005685,27.162673,0.919049,0.447559,"[0.09299166666666667, 0.48033333333333333, 0.9...","[0.1, 0.5, 0.9]"
7,False,1,0.0010,0.005715,27.140217,0.920004,0.447371,"[0.046691666666666666, 0.3932083333333333, 0.9...","[0.1, 0.5, 0.9]"
8,False,2,0.0010,0.005593,27.234100,0.921194,0.452003,"[0.08941666666666667, 0.47265833333333335, 0.9...","[0.1, 0.5, 0.9]"
9,False,3,0.0010,0.005534,27.280044,0.921534,0.456579,"[0.06260833333333334, 0.420475, 0.910566666666...","[0.1, 0.5, 0.9]"
